# Static Stress & Strain — Hand Calculations

Executable validation formulas for static structural analysis.
Enter your ANSYS results and geometry parameters to compute analytical solutions and error.

**Reference:** `validation.md` — Static Stress & Strain, Buckling, Beam Deflection, Pressure Vessels

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def error_pct(ansys_val, hand_calc):
    """Percent error between ANSYS result and hand calculation."""
    return abs(ansys_val - hand_calc) / abs(hand_calc) * 100

def print_comparison(name, ansys_val, hand_calc, unit, threshold=5.0):
    """Print ANSYS vs hand calc comparison with pass/fail."""
    err = error_pct(ansys_val, hand_calc)
    status = 'PASS' if err < threshold else 'FAIL'
    print(f'{name}:')
    print(f'  Hand calc:  {hand_calc:.4f} {unit}')
    print(f'  ANSYS:      {ansys_val:.4f} {unit}')
    print(f'  Error:      {err:.2f}% [{status} — threshold {threshold}%]')
    print()

---
## 1. Axial Stress: σ = F/A

In [ ]:
# --- INPUT ---
F = 10000       # Applied force [N]
A = 0.001       # Cross-sectional area [m²]
E = 200e9       # Young's modulus [Pa]

ansys_stress = 0.0   # <-- Enter your ANSYS max stress [Pa]
ansys_strain = 0.0   # <-- Enter your ANSYS max strain [-]

# --- HAND CALC ---
sigma = F / A                # Normal stress [Pa]
epsilon = sigma / E          # Hooke's law strain [-]

print(f'σ = F/A = {F}/{A} = {sigma/1e6:.2f} MPa')
print(f'ε = σ/E = {sigma:.0f}/{E:.0e} = {epsilon:.6f}')
print()

if ansys_stress > 0:
    print_comparison('Stress', ansys_stress/1e6, sigma/1e6, 'MPa')
if ansys_strain > 0:
    print_comparison('Strain', ansys_strain, epsilon, '—')

---
## 2. Bending Stress: σ = My/I

In [ ]:
# --- INPUT ---
M = 500         # Bending moment [N·m]
b = 0.05        # Beam width [m]
h = 0.1         # Beam height [m]

ansys_bending_stress = 0.0   # <-- Enter ANSYS result [Pa]

# --- HAND CALC ---
I = b * h**3 / 12            # Moment of inertia [m⁴]
y = h / 2                    # Distance to outer fiber [m]
sigma_b = M * y / I          # Bending stress [Pa]

print(f'I = bh³/12 = {I:.6e} m⁴')
print(f'σ = My/I = {M}×{y}/{I:.6e} = {sigma_b/1e6:.2f} MPa')
print()

if ansys_bending_stress > 0:
    print_comparison('Bending stress', ansys_bending_stress/1e6, sigma_b/1e6, 'MPa')

---
## 3. Beam Deflection

In [ ]:
# --- INPUT ---
P = 1000        # Applied load [N]
L = 1.0         # Beam length [m]
E = 200e9       # Young's modulus [Pa]
I = 8.333e-6    # Moment of inertia [m⁴] (or compute from b, h above)

ansys_deflection = 0.0   # <-- Enter ANSYS max deflection [m]

# --- HAND CALC ---
# Choose the appropriate formula:
delta_cantilever = P * L**3 / (3 * E * I)       # Cantilever, tip load
delta_ss_center = P * L**3 / (48 * E * I)       # Simply supported, center load

print('Cantilever (tip load):')
print(f'  δ = PL³/3EI = {delta_cantilever*1000:.4f} mm')
print()
print('Simply supported (center load):')
print(f'  δ = PL³/48EI = {delta_ss_center*1000:.4f} mm')
print()

if ansys_deflection > 0:
    # Compare to whichever formula matches your BC
    print_comparison('Deflection (cantilever)', ansys_deflection*1000, delta_cantilever*1000, 'mm')

---
## 4. Euler Buckling: P_cr = π²EI/(KL)²

In [ ]:
# --- INPUT ---
E = 200e9       # Young's modulus [Pa]
I = 8.333e-6    # Moment of inertia [m⁴]
L = 2.0         # Column length [m]
K = 1.0         # Effective length factor (1.0 = pinned-pinned, 0.5 = fixed-fixed, 2.0 = fixed-free)

ansys_buckling_load = 0.0   # <-- Enter ANSYS eigenvalue buckling load [N]

# --- HAND CALC ---
P_cr = np.pi**2 * E * I / (K * L)**2

print(f'P_cr = π²EI/(KL)² = {P_cr:.0f} N = {P_cr/1000:.2f} kN')
print(f'  K = {K} (effective length factor)')
print()

if ansys_buckling_load > 0:
    print_comparison('Buckling load', ansys_buckling_load/1000, P_cr/1000, 'kN', threshold=10.0)

---
## 5. Pressure Vessel: σ_h = pr/t

In [ ]:
# --- INPUT ---
p = 5e6         # Internal pressure [Pa]
r = 0.5         # Mean radius [m]
t = 0.01        # Wall thickness [m]

ansys_hoop = 0.0     # <-- Enter ANSYS hoop stress [Pa]
ansys_axial = 0.0    # <-- Enter ANSYS axial stress [Pa]

# --- HAND CALC ---
sigma_h = p * r / t          # Hoop (circumferential) stress
sigma_a = p * r / (2 * t)    # Axial (longitudinal) stress

print(f'Thin-wall check: r/t = {r/t:.1f} (must be > 10 for thin-wall)')
print(f'σ_hoop = pr/t = {sigma_h/1e6:.1f} MPa')
print(f'σ_axial = pr/2t = {sigma_a/1e6:.1f} MPa')
print()

if ansys_hoop > 0:
    print_comparison('Hoop stress', ansys_hoop/1e6, sigma_h/1e6, 'MPa')
if ansys_axial > 0:
    print_comparison('Axial stress', ansys_axial/1e6, sigma_a/1e6, 'MPa')

---
## 6. Weld Shear Stress: τ = F/(0.707·h·l)

In [ ]:
# --- INPUT ---
F = 20000       # Applied force [N]
h = 0.006       # Weld leg size [m]
l = 0.1         # Weld length [m]

ansys_weld_stress = 0.0   # <-- Enter ANSYS stress at weld throat [Pa]

# --- HAND CALC ---
throat = 0.707 * h         # Throat thickness
A_weld = throat * l        # Throat area
tau = F / A_weld           # Shear stress on weld

print(f'Throat = 0.707 × {h*1000:.1f} mm = {throat*1000:.2f} mm')
print(f'A_weld = {A_weld*1e6:.1f} mm²')
print(f'τ = F/A_weld = {tau/1e6:.1f} MPa')
print()

# Allowable check (E70 electrode)
S_ut_electrode = 482e6     # E70xx electrode tensile strength [Pa]
tau_allow = 0.30 * S_ut_electrode
SF = tau_allow / tau
print(f'Allowable τ (E70) = 0.30 × {S_ut_electrode/1e6:.0f} = {tau_allow/1e6:.1f} MPa')
print(f'Safety factor = {SF:.2f}')
print()

if ansys_weld_stress > 0:
    print_comparison('Weld shear stress', ansys_weld_stress/1e6, tau/1e6, 'MPa')

---
## 7. Shaft Torsion: θ = TL/GJ

In [ ]:
# --- INPUT ---
T = 200         # Applied torque [N·m]
L = 0.5         # Shaft length [m]
d = 0.05        # Shaft diameter [m]
G = 79.3e9      # Shear modulus [Pa] (steel ≈ 79.3 GPa)

ansys_twist = 0.0       # <-- Enter ANSYS twist angle [rad]
ansys_shear = 0.0       # <-- Enter ANSYS max shear stress [Pa]

# --- HAND CALC ---
J = np.pi * d**4 / 32          # Polar moment of inertia [m⁴]
theta = T * L / (G * J)        # Angle of twist [rad]
tau_max = T * (d/2) / J        # Max shear stress at surface [Pa]

print(f'J = πd⁴/32 = {J:.6e} m⁴')
print(f'θ = TL/GJ = {theta:.6f} rad = {np.degrees(theta):.4f}°')
print(f'τ_max = Tc/J = {tau_max/1e6:.2f} MPa')
print()

if ansys_twist > 0:
    print_comparison('Twist angle', np.degrees(ansys_twist), np.degrees(theta), 'deg')
if ansys_shear > 0:
    print_comparison('Max shear stress', ansys_shear/1e6, tau_max/1e6, 'MPa')